# Project 12 - Decision Tree - using Python
## Nhi Le
## DATA 4319 

**Decision tree** builds classification or regression models in the form of a tree structure. It breaks down a dataset into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed. The final result is a tree with decision nodes and leaf nodes. A decision node (e.g., Outlook) has two or more branches (e.g., Sunny, Overcast and Rainy). Leaf node (e.g., Play) represents a classification or decision. The topmost decision node in a tree which corresponds to the best predictor called root node. Decision trees can handle both categorical and numerical data. 

![](https://www.saedsayad.com/images/Decision_Tree_1.png)

**Algorithm**

Decision Tree algorithm belongs to the family of supervised learning algorithms. Unlike other supervised learning algorithms, the decision tree algorithm can be used for solving regression and classification problems too.

The goal of using a Decision Tree is to create a training model that can use to predict the class or value of the target variable by learning simple decision rules inferred from prior data(training data).

In Decision Trees, for predicting a class label for a record we start from the root of the tree. We compare the values of the root attribute with the record’s attribute. On the basis of comparison, we follow the branch corresponding to that value and jump to the next node.

**Types of Decision Trees**
 
Types of decision trees are based on the type of target variable we have. It can be of two types:

* _Categorical Variable Decision Tree:_ Decision Tree which has a categorical target variable then it called a Classification

* _Continuous Variable Decision Tree:_ Decision Tree has a continuous target variable then it is called Regression

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
def split(array):
    dic={}
    for i in np.unique(array):
        dic.update({i:np.where(array==i)[0]})
    return dic

In [5]:
print(split(np.array([0,1,2])))
print(split(np.array([1,0,1,0,0,1,0])))
print(split(np.array([1,0,3,2,0,1,1])))

{0: array([0], dtype=int64), 1: array([1], dtype=int64), 2: array([2], dtype=int64)}
{0: array([1, 3, 4, 6], dtype=int64), 1: array([0, 2, 5], dtype=int64)}
{0: array([1, 4], dtype=int64), 1: array([0, 5, 6], dtype=int64), 2: array([3], dtype=int64), 3: array([2], dtype=int64)}


## Entropy

**Entropy** controls how a Decision Tree decides to split the data. It actually effects how a Decision Tree draws its boundaries. Entropy is the measures of impurity, disorder or uncertainty in a bunch of examples.

The Mathematical formula for Entropy is as follows:

![](https://miro.medium.com/max/587/1*nNY_7_aWRwp8E2DyGduEPg.png)
![](https://miro.medium.com/max/3600/1*S6zcbdAzUvIOKBaWBKp9MA.png)

In [6]:
import math
def entropy(array):
    b_list=[]
    for i in np.unique(array):
        p=len(np.where(array==i)[0])/len(array)
        b_list.append(p*math.log2(p))
    return -sum(b_list)

In [7]:
print(round(entropy(np.array([0,1,0,1,1,0])),4))
print(round(entropy(np.array([1,2])),4))
print(round(entropy(np.array([1,1])),4))
print(round(entropy(np.array([1,0,0,0,0,0,0,0,0,0,0])),4))
print(round(entropy(np.array([0,0,0])),4))
print(round(entropy(np.array([1,1,1,0,1,4,4,2,1])),4))

1.0
1.0
-0.0
0.4395
-0.0
1.6577


## Information gain (IG)

**Information gain (IG)** measures how much “information” a feature gives us about the class.

![](https://miro.medium.com/max/3600/1*bVGWGETTor7bSnhr7sXEVw.png)

**Why it matter ?**
* Information gain is the main key that is used by Decision Tree Algorithms to construct a Decision Tree.

* Decision Trees algorithm will always tries to maximize Information gain.

* An attribute with highest Information gain will tested/split first.

In [8]:
def IG(x,y):
    parent_entropy=entropy(y)
    split_dict=[split(x).get(k) for k in np.unique(x)]
    for i in split_dict:
        freq=len(x[[k for k in i]])/len(x)
        child_entropy=freq*entropy(y[[k for k in i]])
        parent_entropy=parent_entropy-child_entropy
    return parent_entropy

In [9]:
x=np.array([0,1,0,1,0,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,0,1,1,2,2])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))
x=np.array([0,1,0,1,2,1])
y=np.array([0,1,0,1,1,1])
print(round(IG(x,y),4))

0.4591
0.2516
0.9183


In [10]:
def make_tree(X,y,attribute):
    if y.shape[0]==1 or y.shape[0]==0:
        return y

    gains=[]
    if len(X.T)==1:
        gain=IG(X.T,y)
        if (gain<=1e-05):
            return y
        gains.append(gain)
    else:
        for x in X.T:
            gain=IG(x,y)

            if (gain<=1e-05):
                return y

            gains.append(gain)
    #print(gains)
    best_feature=np.argmax(gains)
    #print(best_feature)
    results={}
    
    subset_dict=split(X[:,best_feature])
    #print(subset_dict)
    for feature_value,train_example_indices in subset_dict.items():
        #print(train_example_indices)
        child_x_subset=np.delete(X[train_example_indices],best_feature,1)
        child_y_subset=y[train_example_indices]
        child_attribute=attribute[attribute != attribute[best_feature]]
        #print(child_x_subset)

        results["%s = %s" %(attribute[best_feature], feature_value)] = make_tree(child_x_subset, child_y_subset,child_attribute)

    return results

In [11]:
x=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].values
y=pd.read_csv("dataset/tennis.csv").iloc[:,-1].values
attribute=pd.read_csv("dataset/tennis.csv").iloc[:,1:-1].columns.values

### Decision Tree model for dataset Tennis

![img](https://scontent.xx.fbcdn.net/v/t1.15752-9/129871522_1049163618884219_2704732472935006068_n.png?_nc_cat=106&ccb=2&_nc_sid=ae9488&_nc_ohc=2RaH9P7stxgAX_K73tM&_nc_ad=z-m&_nc_cid=0&_nc_ht=scontent.xx&oh=25e2e5dcb03975620c1d92794161bf32&oe=5FF3A1C6)

In [12]:
golf_df=pd.read_csv("dataset/tennis.csv").iloc[:,1:]
golf_df

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [13]:
print(attribute)
print(x)
print("label\n",y)
print("decision tree:\n",make_tree(x,y,attribute))

['outlook' 'temp' 'humidity' 'wind']
[['Sunny' 'Hot' 'High' 'Weak']
 ['Sunny' 'Hot' 'High' 'Strong']
 ['Overcast' 'Hot' 'High' 'Weak']
 ['Rain' 'Mild' 'High' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Cool' 'Normal' 'Strong']
 ['Overcast' 'Cool' 'Normal' 'Strong']
 ['Sunny' 'Mild' 'High' 'Weak']
 ['Sunny' 'Cool' 'Normal' 'Weak']
 ['Rain' 'Mild' 'Normal' 'Weak']
 ['Sunny' 'Mild' 'Normal' 'Strong']
 ['Overcast' 'Mild' 'High' 'Strong']
 ['Overcast' 'Hot' 'Normal' 'Weak']
 ['Rain' 'Mild' 'High' 'Strong']]
label
 ['No' 'No' 'Yes' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes' 'Yes' 'Yes' 'Yes' 'Yes'
 'No']
decision tree:
 {'outlook = Overcast': array(['Yes', 'Yes', 'Yes', 'Yes'], dtype=object), 'outlook = Rain': {'wind = Strong': array(['No', 'No'], dtype=object), 'wind = Weak': array(['Yes', 'Yes', 'Yes'], dtype=object)}, 'outlook = Sunny': {'humidity = High': array(['No', 'No', 'No'], dtype=object), 'humidity = Normal': array(['Yes', 'Yes'], dtype=object)}}


In [14]:
def _traverse(x,d,attribute):
    if isinstance(d,np.ndarray):
        return d
    for key in d:
        name,value=key.split("=")
        feature_idx=attribute.tolist().index(name.strip())
        if x[feature_idx]==value.strip():
            return _traverse(x,d[key],attribute)

In [15]:

_traverse(np.array(['Rain','Mild','High','Weak']),make_tree(x,y,attribute),attribute)

array(['Yes', 'Yes', 'Yes'], dtype=object)

the answer is "Yes"

In [18]:
_traverse(np.array(['Overcast','Cool','High','Weak']),make_tree(x,y,attribute),attribute)

array(['Yes', 'Yes', 'Yes', 'Yes'], dtype=object)

the answer is "Yes"

In [17]:
for i,j in zip(golf_df.iloc[:,:-1].values,golf_df['play']):
    print("predict:",_traverse(i,make_tree(x,y,attribute),attribute))
    print('lable:',j)

predict: ['No' 'No' 'No']
lable: No
predict: ['No' 'No' 'No']
lable: No
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No']
lable: No
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No' 'No']
lable: No
predict: ['Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['Yes' 'Yes' 'Yes' 'Yes']
lable: Yes
predict: ['No' 'No']
lable: No


### Conclusion: 
All prediction is correct. Decision trees use multiple algorithms to decide to split a node into two or more sub-nodes. The creation of sub-nodes increases the homogeneity of resultant sub-nodes. In other words, we can say that the purity of the node increases with respect to the target variable. The decision tree splits the nodes on all available variables and then selects the split which results in most homogeneous sub-nodes.

**Reference:** https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01